![](../nci-logo.png)

-------
# Data Access and Manipulation Using iPython Notebooks
## Working with HDF and netCDF files



### In this notebook:

- Using iPython Notebooks with NetCDF and h5 files within the VDI
    - <a href='#part1'>Launch Jupyter Notebook</a>  
    - <a href='#part2'>Importing Python libraries</a>  
    - <a href='#part3'>How to write HDF and netCDF files</a> 
    - <a href='#part4'>Opening file and viewing contents</a>
    - <a href='#part5'>Command-line tools for HDF5 files</a>
    - <a href='#part6'>NCO basics</a>
    
---------

<br>

<a id='part1'></a> 
## Launch the Jupyter Notebook application

#### Using pre-built VDI modules:

Load the following modules:

```
    $ module load python/2.7.11
    $ module load python/2.7.11-matplotlib
    $ module load ipython/4.2.0-py2.7
    $ module load netcdf/4.3.3.1
    $ module load netcdf4-python/1.2.4-ncdf-4.3.3.1-py2.7
    $ module load h5py/2.6.0-hdf5-1.8.14p-py2.7
    $ module load nco/4.5.3
```    
    
<br>
Launch the Jupyter Notebook application:
```
    $ jupyter notebook
``` 

<div class="alert alert-info">
<b>NOTE: </b> This will launch the <b>Notebook Dashboard</b> within a new web browser window. 
</div>

<br>

#### Using virtual environments:

To use along with customised python packages in a virtual environment, begin by following the steps in **Python on the VDI: Part II**. 

Once you have a virtual environment setup with your packages (including `Jupyter`), proceed by loading the required modules and activating the virtual environment:

```
    $ module load python/2.7.11
    $ source <path_to_virtual_environment>/bin/activate
```

<br>
Then, as above, launch the Jupyter Notebook application:

```
    $ jupyter notebook
```    
    
<div class="alert alert-warning">
<b>NOTE: </b> If you have already followed <b>Python on the VDI: Part II</b>, you should have installed the netcdf4-python package, which is required in the remainder of this notebook.  
</div>

<br>


<br>


<a id='part2'></a> 
## Import python modules

In [ ]:
import h5py
import numpy as np
from netCDF4 import Dataset
import time
from numpy.random import uniform

<a id='part3'></a> 
## How to write HDF5 and NetCDF files

### HDF (Heirarchical Data Format)

Let's first work on creating a HDF5 (Heirarchical Data Format) file. HDF5 is a powerful binary data format with no upper limit on the file size. It provides parallel IO and runs under the hood low level optimisations to make queries faster and storage requirements smaller. HDF5 files work generally like standard Python file objects. They support standard modes like r/w/a, and should be closed when they are no longer in use.

We have to initialise our HDF5 file using <span style="color:red">h5py.File</span> and providing the arguments of filename and mode. As we are writing this file, we provide a __w__ for write access:

In [ ]:
h5file = h5py.File("testfile.hdf5", "w")

The file name may be a byte string or unicode string. 

| Valid modes        | Description           |
| ------------- |:-------------:|
| r   | Readonly, file must exist |
| r+      | Read/write, file must exist      |
| w | Create file, truncate if exists      |
| w- or x | Create file, fail if exists |
| a | Read/write if exists, create otherwise (default)|

Now we can create an HDF5 dataset. Datasets are very similar to NumPy arrays in that they are homogeneous collections of data elements, with an immutable datatype and (hyper)rectangular shape. HDF5 datasets support a variety of transparent storage features (e.g. compression, error-dection and chunked I/O).

New datasets are created using either <span style="color:red">Group.create_dataset()</span> or <span style="color:red">Group.require_dataset()</span>. To make an empty dataset, one must specify a name, shape and (optionally) the data type (default is 'f'):

In [ ]:
 dset1 = h5file.create_dataset("Zxx", (1000,), dtype='i')

HDF5 datasets have both a shape and data type:

In [ ]:
dset1.shape

In [ ]:
dset1.dtype

It is possible to initialise the dataset to an existing NumPy array:

In [ ]:
initdata = np.arange(0,1,0.001)

In [ ]:
dset2 = h5file.create_dataset("Zxy", data=initdata)

In [ ]:
dset3 = h5file.create_dataset("Zyx", data=initdata, dtype='f')

HDF5 datasets are by default contiguous. However, datasets can be created using HDF5's chunked storage layout. This means the dataset is divided up into regularly-sized pieces which are stored haphazardly on disk, and indexed using a B-tree.

Chunked storage makes it possible to resize datasets, allowing compression filters. To enable chunked storage, set the keyword <span style="color:red">chunk</span> to a tuple indicating the chunk shape


In [ ]:
dset4 = h5file.create_dataset("Zyy", (1000,1000), chunks=(100,100))

Here, data will be read and written in blocks with shape (100,100). Since picking a chunk shape can be confusing, h5py can guess a chunk shape for you

In [ ]:
dset5 = h5file.create_dataset("Zyyb", (1000,1000), chunks=True)

In [ ]:
print dset1.shape, dset2.shape, dset3.shape, dset4.shape, dset5.shape

### Groups and heirarchical organisation

Every object in an HDF5 file has a name, and they're arranged in a POSIX-style hierarchy. 

In [ ]:
dset3.name

The "folders" in this system are called <span style="color:red">groups</span>. Let's now create a subgroup using <span style="color:red">create_group</span> and add some other datasets to this subgroup.

In [ ]:
group1 = h5file.create_group("seismic")

In [ ]:
dsetseis1 = group1.create_dataset("some_seismic_data", (50,), dtype = 'f')

In [ ]:
dsetseis1.name

You can specify the full path when creating the datasets:

In [ ]:
dataseis2 = h5file.create_dataset("subgroup2/some_more_seismic_data", (10,), dtype='i')

In [ ]:
dataseis1a = h5file.create_dataset("seismic/some_more_seismic", (10,), dtype='i')

In [ ]:
dataseis1a.name

We can retrieve objects in the file using the item-relevant syntax:

In [ ]:
myfileseis = h5file['seismic/some_more_seismic']

Let's list all the groups in our file

In [ ]:
for name in h5file:
    print name

One great feature of HDF5 is that you can store metadata right next to the data it describes. All groups and datasets support attached named bits of data called *attributes*:

In [ ]:
dset1.attrs['latitude'] = 32.2

In [ ]:
dset1.attrs['longitude'] = 144.2

In [ ]:
'longitude' in dset1.attrs

All we need to do now is close the file, which will write all our work to disk

In [ ]:
h5file.close()

### NetCDF (Network Common Data Form)

Now let's create a netCDF file. For this we are going to use the netCDF4-python module. To create a netCDF file from python, use the  <span style="color:red">Dataset()</span> constructor. This method is also used to open an existing netCDF file.

In [ ]:
dataset_nCDF = Dataset('geophys.nc','w',format='NETCDF4_CLASSIC')

In [ ]:
print dataset_nCDF.file_format

Define a set of dimensions used for your variables:

In [ ]:
period = dataset_nCDF.createDimension('period', 20)
apparent_resistivity = dataset_nCDF.createDimension('apparent_resistivity',503)
lat = dataset_nCDF.createDimension('lat', 73)
lon = dataset_nCDF.createDimension('lon', 144)
time = dataset_nCDF.createDimension('time', None)

In [ ]:
print len(lon), len(time)

__Dimensions__: All of the *Dimension* instances are stored in a python dictionary. Therefore, we can access each dimension by its name using dictionary key access: 

In [ ]:
print 'Lon dimension:', dataset_nCDF.dimensions['lon']
print 'Lat dimension:', dataset_nCDF.dimensions['lat']
print 'Period dimension:', dataset_nCDF.dimensions['period']

In [ ]:
for dimname in dataset_nCDF.dimensions.keys():
    dim = dataset_nCDF.dimensions[dimname]
    print dimname, len(dim), dim.isunlimited()

__Variables__: NetCDF variables behave much like python multi-dimensional arrays in numpy. However, unlike numpy arrays, netCDF variables can be appended to along the *unlimited* dimension. To create a netCDF variable, use <span style="color:red">Dataset.createVariable(*var_id*, *type*, *dimensions*)</span>:  

In [ ]:
times = dataset_nCDF.createVariable('time', np.float64, ('time',))
latitudes = dataset_nCDF.createVariable('latitude', np.float32, ('lat',))
longitudes = dataset_nCDF.createVariable('longitude', np.float32, ('lon',))
periods = dataset_nCDF.createVariable('period', np.float32, ('period',))

# create the actual 4-D variable
app_resistivities = dataset_nCDF.createVariable('apparent_resistivity', np.float32, ('time','lat','lon','period'))


In [ ]:
app_resistivities

All of the variables in the *Dataset* are stored in a Python dictionary:

In [ ]:
print 'lat variable:', dataset_nCDF.variables['latitude']

In [ ]:
for varname in dataset_nCDF.variables.keys():
    var = dataset_nCDF.variables[varname]
    print varname, var.dtype, var.dimensions, var.shape

__Attributes (global)__: Global attributes are set by assigning values to *Dataset* instance variables. Attributes can be strings, numbers or sequences.

__Attributes (variable)__: Variable attributes are set by assigning to *Variable* instance variables:

In [ ]:
# Global Attributes

dataset_nCDF.description = 'some test EM data'
dataset_nCDF.history = 'Created'+time.ctime(time.time())
dataset_nCDF.source = 'netCDF4 python module tutorial'

In [ ]:
# Variable attributes

latitudes.units='degree_north'
longitudes.units='degree_east'
times.units='hours since 0001-01-01 00:00:00'
times.calendar='gregorian'
periods.units='seconds'
app_resistivities.units='ohm.m'

In [ ]:
print dataset_nCDF.description
print dataset_nCDF.history

__Writing Data__: To put data into our netCDF Variables, we can assign data to a slice:

In [ ]:
lats = np.arange(-90,90,2.5)
lons = np.arange(-180,180,2.5)
periods = np.arange(0.01,100,5)
latitudes[:] = lats
longitudes[:] = lons
periods[:] = periods

In [ ]:
print 'latitudes =\n', latitudes[:]
print 'longitudes =\n', longitudes[:]

NetCDF *variable* objects that have an unlimited dimension will grow along that dimension if you assign data outside the currently defined range of indices:

In [ ]:
print 'app_resistivity shape before adding data =', app_resistivities.shape

In [ ]:
nlats = len(dataset_nCDF.dimensions['lat'])
nlons = len(dataset_nCDF.dimensions['lon'])
nperiods = len(dataset_nCDF.dimensions['period'])
app_resistivities[1:10,:,:,:] = uniform(size=(9,nlats,nlons,nperiods))

__Time coordinates__: Most metadata standards (e.g. CF, COARDS) specify that time be measured relative to a fixed date using a certain calendar (e.g. hours since YY:MM:DD hh-mm-ss"). The functions <span style="color:red">num2date()</span> and <span style="color:red">date2num()</span> can be used to convert values to and from calendar dates:

In [ ]:
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num

In [ ]:
dates = []

for n in range(app_resistivities.shape[0]):
    dates.append(datetime(2000,1,1)+ n*timedelta(hours=12))

times[:] = date2num(dates, units = times.units, calendar = times.calendar)

In [ ]:
print 'time values (in units %s):'% times.units + '\n', times[:]

In [ ]:
dates = num2date(times[:], units = times.units, calendar=times.calendar)

In [ ]:
print 'dates corresponding to time values:\n', dates

__Finally, we need to write the file:__

In [ ]:
dataset_nCDF.close()

and the file is written.

<a id='part4'></a> 
## Opening file and viewing contents

Let's first begin with a HDF file. For this example we will use the same file we previously created. To open and read data, use the <span style="color:red">File</span> method in read mode, *r*:

In [ ]:
myfile = h5py.File('testfile.hdf5','r')

Let's see what data is in the file by using the call <span style="color:red">keys()</span> on the file object:

In [ ]:
myfile.keys()

We can grab some of the datasets we created using the <span style="color:red">get</span> method and specifying the dataset name:

In [ ]:
d1 = myfile.get('Zxy')

In [ ]:
d1

In [ ]:
d2 = myfile.get('Zyy')

In [ ]:
d2

We can see all the groups and datasets in our HDF file by running:

In [ ]:
all_groups = [obj for obj in myfile if isinstance(myfile[obj],h5py.Group)]
all_datasets = [obj for obj in myfile if isinstance(myfile[obj],h5py.Dataset)]

In [ ]:
print all_groups, all_datasets

Let's see what exists in the group *seismic*:

In [ ]:
myfile["seismic"].keys()

Now we can grab a dataset within *seismic*:

In [ ]:
d3 = myfile["seismic"].get('some_seismic_data')

In [ ]:
d3

__d1__, __d2__ and __d3__ are HDF5 dataset objects. To convert these into arrays, use numpy's array method:

In [ ]:
d1a = np.array(d1)
d1a.shape

In [ ]:
d2a = np.array(d2)
d2a.shape

In [ ]:
d3a = np.array(d3)
d3a.shape

In [ ]:
d1a

In [ ]:
d2a

In [ ]:
d3a

In [ ]:
myfile.close()

Now let's open and view the contents of the netCDF file we created in the previous section. To open a netCDF file from python, call the <span style="color:red"> Dataset()</span> constructor:

In [ ]:
netcdf_dataset = Dataset('geophys.nc')

In [ ]:
print netcdf_dataset.file_format

Interrogate dimensions:

In [ ]:
print netcdf_dataset.dimensions.keys()

In [ ]:
print netcdf_dataset.dimensions['lat']
print netcdf_dataset.dimensions['time']
print netcdf_dataset.dimensions['apparent_resistivity']

Interrogate variables

In [ ]:
print netcdf_dataset.variables.keys()

In [ ]:
print netcdf_dataset.variables['latitude']
print netcdf_dataset.variables['time']
print netcdf_dataset.variables['apparent_resistivity']

Interrogate global and variable attributes

In [ ]:
# Find all netCDF global attributes

for attr in netcdf_dataset.ncattrs():
    print attr, '=', getattr(netcdf_dataset,attr)

In [ ]:
# Find variable attributes

netcdf_dataset.variables

In [ ]:
netcdf_dataset.close()

<a id='part5'></a> 
## Command-line tools for HDF5 files

There are numerous command-line tools included in the HDF5 distribution to view, edit, convert and compare HDF5 files. Let's use our testfile.hdf5 file for the following examples. We will begin with <span style="color:red">h5dump</span>, which enables the user to examine the contents of an HDF5 and dump those contents to an ASCII file: 

In [ ]:
h5dump -n testfile.hdf5 # -n displays a list of the objects in a file

h5dump -H testfile.hdf5 # displays the header information only (no data)

h5dump -d "/seismic/some_seismic_data" testfile.hdf5 # display a specific dataset.

h5dump -d "seismic/some_seismic_data" -o seismic.txt -y testfile.hdf5 # converts specified dataset to ASCII file.

<span style="color:red">h5stat</span> can be used to print statistics about HDF5 files:

In [ ]:
h5stat testfile.hdf5

For a complete list of HDF5 tools available, see the HDF5 tools page https://support.hdfgroup.org/HDF5/doc/RM/Tools.html

<a id='part6'></a> 
## NCO basics

__NCO (NetCDF Operators)__ is a suite of command-line based tools designed to facilitate manipulation and analysis of self-describing data stored in the netCDF-accessible formats, including DAP, HDF4 and HDF5. Let's begin with the <span style="color:red">ncks</span> (netCDF kitchen sink) operator. This command can give an overview of a netCDF file, extract certain variables, extract certain dimensions and manipulate record dimensions. Let's use our example geophys.nc file for this example and open up a terminal.    

In [ ]:
# View the contents of a netCDF file

ncks geophys.nc | more

# View a variable

ncks -v latitude geophys.nc | more
ncks -v apparent_resistivity geophys.nc | more

# View multiple variables

ncks -v longitudetime geophys.nc | more

# View a variable over some dimension subsets

ncks -v apparent_resistivity -d lat,5,7 -d period,1,2 -d lon,55,57 geophys.nc | more 

<span style="color:red">ncks</span> can also output data from an input file into an output file:

In [ ]:
ncks -v latitude,longitude geophys.nc -O grid.nc 

In [ ]:
ncks -v apparent_resistivity -d lat,5,7 -d period,1,2 -d lon,55,57 geophys.nc -O appres.nc

Now we can look at the <span style="color:red">ncrcat</span> and <span style="color:red">ncecat</span> commands. These concatinate multiple files together into a single file. Use <span style="color:red">ncrcat</span> when there is a record dimension (e.g. concatinating multiple daily files into one monthly file) and <span style="color:red">ncecat</span> when there in no record dimension - a new record dimension will be created.

In [ ]:
ncrcat file1 file2 -O outputfile

Test <span style="color:red">ncrcat</span> and <span style="color:red">ncecat</span> on some files you are interested in.

Other operators include:
- __ncap2__: netCDF Arithmetic Processor
- __ncatted__: netCDF Attribute Editor
- __ncbo__: netCDF Binary Operator
- __nces__: netCDF Ensemble Statistics
- __ncflint__: netCDF File Interpolator
- __ncra__: netCDF Record Averager
- __ncwaa__: netCDF Weighted Averager

Try these operators out on your netCDF files! For help on a particular operator, type <span style="color:red"> man operator </span> (e.g. man ncbo)